# Kaggle Competition : Microsoft Malware Data
> This is a Sungryong Hong's Notebook.  

> I have a stand-alone Spark(2.3.2)/Hadoop(2.8.3) cluster, which has 48 logical cores with 150GB memory. 

> I have put the data files to my hdfs. Check the contents as `hfs -cat /data/spark/msmalware/test.csv | head`.  

>`hfs` is an alias for `hdfs dfs`. 



## 1. Import Basic Packages

In [1]:
# Basic Libraries 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import gc

pd.set_option('display.max_rows', 500)

# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

#### Spark-Shell Sesssion 

In [2]:
# Basic PySpark Libraries

# Old Style : SparkContext 
#from pyspark import SparkContext   
#from pyspark.sql import SQLContext


# New Style : Spark Session  
#Shell-Mode: Spark Session Name is `spark`

sc = spark.sparkContext
sqlsc = SQLContext(sc)
sc.setCheckpointDir("hdfs://master:54310/tmp/spark/checkpoints")

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

In [3]:
# Enable Arrow for boosting up python performances 
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set('spark.debug.maxToStringFields',50)

## 2. Read `mldata`

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq

In [5]:
mldata = sqlsc.read.parquet('hdfs://master:54310/data/spark/msmalware/mldata.parquet.snappy')

In [6]:
print mldata.columns

['MachineIdentifier', 'IsBeta', 'IsSxsPassiveMode', 'HasTpm', 'CountryIdentifier', 'LocaleEnglishNameIdentifier', 'OsBuild', 'OsSuite', 'AutoSampleOptIn', 'Census_HasOpticalDiskDrive', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSUILocaleIdentifier', 'Census_IsPortableOperatingSystem', 'Census_IsSecureBootEnabled', 'Census_IsTouchEnabled', 'Census_IsPenCapable', 'HasDetections', 'ProductName_indexed_onehot', 'EngineVersion_indexed_onehot', 'AppVersion_indexed_onehot', 'AvSigVersion_indexed_onehot', 'RtpStateBitfield_indexed_onehot', 'DefaultBrowsersIdentifier_indexed_onehot', 'AVProductStatesIdentifier_indexed_onehot', 'AVProductsInstalled_indexed_onehot', 'AVProductsEnabled_indexed_onehot', 'CityIdentifier_indexed_onehot', 'OrganizationIdentifier_indexed_onehot', 'GeoNameIdentifier_indexed_onehot', 'Platform_indexed_onehot', 'Processor_indexed_onehot', 'OsVer_indexed_onehot', 'OsPlatformSubRelease_indexed_onehot', 'OsBuildLab_indexed_onehot', 'SkuEdition_indexed_onehot'

In [7]:
len(mldata.columns)

84

## 4. Run MLs

In [8]:
mldata.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
+--------------------+
only showing top 5 rows



#### Tiny Sample

In [9]:
%%time
mltiny = mldata.sample(False,0.01,seed=777)

CPU times: user 1.37 ms, sys: 1.11 ms, total: 2.48 ms
Wall time: 27.9 ms


In [10]:
mltiny.cache()

DataFrame[MachineIdentifier: string, IsBeta: int, IsSxsPassiveMode: int, HasTpm: int, CountryIdentifier: int, LocaleEnglishNameIdentifier: int, OsBuild: int, OsSuite: int, AutoSampleOptIn: int, Census_HasOpticalDiskDrive: int, Census_OSBuildNumber: int, Census_OSBuildRevision: int, Census_OSUILocaleIdentifier: int, Census_IsPortableOperatingSystem: int, Census_IsSecureBootEnabled: int, Census_IsTouchEnabled: int, Census_IsPenCapable: int, HasDetections: int, ProductName_indexed_onehot: vector, EngineVersion_indexed_onehot: vector, AppVersion_indexed_onehot: vector, AvSigVersion_indexed_onehot: vector, RtpStateBitfield_indexed_onehot: vector, DefaultBrowsersIdentifier_indexed_onehot: vector, AVProductStatesIdentifier_indexed_onehot: vector, AVProductsInstalled_indexed_onehot: vector, AVProductsEnabled_indexed_onehot: vector, CityIdentifier_indexed_onehot: vector, OrganizationIdentifier_indexed_onehot: vector, GeoNameIdentifier_indexed_onehot: vector, Platform_indexed_onehot: vector, Pro

In [11]:
%%time
mltiny.count()

CPU times: user 12.4 ms, sys: 6.83 ms, total: 19.3 ms
Wall time: 1min 9s


89525

In [12]:
mltiny.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
|(383218,[2,3,4,5,...|
+--------------------+
only showing top 5 rows



#### Small Sample

In [81]:
%%time
mlsmall = mldata.sample(False,0.1,seed=888)

CPU times: user 36.1 ms, sys: 36.5 ms, total: 72.5 ms
Wall time: 76.2 ms


In [82]:
%%time
mlsmall.count()

CPU times: user 4.89 ms, sys: 3.62 ms, total: 8.51 ms
Wall time: 2.8 s


892286

### 4.1 Trying `LightGBM` from mmlspark

> Website : https://github.com/Azure/mmlspark/blob/master/docs/lightgbm.md

In [15]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator as BCE

In [16]:
from mmlspark import LightGBMClassifier

In [17]:
gbclassifier = LightGBMClassifier(learningRate=0.3, numIterations=100,\
                                  #earlyStoppingRound=10,\
                                  labelCol='HasDetections',featuresCol='features')

In [18]:
paramGrid = ParamGridBuilder().addGrid(gbclassifier.numLeaves, [10, 20, 40]).build()
gbeval = BCE(labelCol='HasDetections',metricName='areaUnderROC')

In [19]:
gbcrossval = CrossValidator(estimator=gbclassifier, estimatorParamMaps=paramGrid,\
                            evaluator=gbeval,numFolds=4) 

#### For `mltiny`

In [20]:
%%time
gbcvmodeltiny  = gbcrossval.fit(mltiny)

CPU times: user 1.27 s, sys: 371 ms, total: 1.64 s
Wall time: 21min 8s


In [21]:
print("trained LightGBM :%s" % gbcvmodeltiny) #using the best model among cross-validation folds 

trained LightGBM :CrossValidatorModel_415ba520c48b5b68be64


In [22]:
gbcvmodeltiny.avgMetrics

[1.0, 1.0, 1.0]

In [23]:
# display CV score
auc_roc = gbcvmodeltiny.avgMetrics[0]
print("AUC ROC = %g" % auc_roc)
gini = (2 * auc_roc - 1)
print("GINI ~=%g" % gini)

AUC ROC = 1
GINI ~=1


### 4.2 Trying `RandomForestClassifer` from Spark ML

In [32]:
from pyspark.ml.classification import RandomForestClassifier

In [33]:
rfclassifier = RandomForestClassifier(labelCol='HasDetections',featuresCol='features')

In [34]:
rfparamGrid = ParamGridBuilder().addGrid(rfclassifier.numTrees, [10, 30, 50]).build()
rfeval = BCE(labelCol='HasDetections',metricName='areaUnderROC')

In [35]:
rfcrossval = CrossValidator(estimator=rfclassifier, estimatorParamMaps=rfparamGrid,\
                            evaluator=rfeval,numFolds=4) 

#### For `mltiny`

In [36]:
%%time
rfcvmodeltiny  = rfcrossval.fit(mltiny)

CPU times: user 15.5 s, sys: 9.02 s, total: 24.6 s
Wall time: 2h 45min 53s


In [37]:
rfcvmodeltiny.avgMetrics

[0.5710660200474003, 0.6549259685679865, 0.7057613184732777]

> Is `lightGBM` too good? 

### 4.3 Feature Importance 

> `LightGBM` from `mmlspark` shows some error messages when getting the feautre importances  `model.getFeatureImportances()`. Let's see the feature importances from `RandomForestClassifier`. 

In [46]:
# Covert the Vector-assembled feature importances to human-readables 
def ExtractFeatureImportance(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

#### Random Forest 

In [66]:
rfFeatureImportance = ExtractFeatureImportance(rfcvmodeltiny.bestModel.featureImportances,mltiny,'features')

In [70]:
rfFeatureImportance.head(10)

,idx,name,score
383211,383211,Census_PrimaryDiskTotalCapacity_imputed,0.025058
331944,331961,Census_ActivationChannel_indexed_onehot_Volume...,0.022965
39719,39736,AVProductsEnabled_indexed_onehot_1,0.021536
383202,9,Census_OSBuildNumber,0.021438
147497,147514,OsPlatformSubRelease_indexed_onehot_rs4,0.020560
11,28,EngineVersion_indexed_onehot_1.1.14800.3,0.020365
148487,148504,SmartScreen_indexed_onehot_ExistsNotSet,0.019666
39711,39728,AVProductsInstalled_indexed_onehot_1,0.018923
76,93,AppVersion_indexed_onehot_4.18.1807.18075,0.016222
147511,147528,OsBuildLab_indexed_onehot_16299.15.x86fre.rs3_...,0.016156


#### LightGBM

In [106]:
# the below command will produce some error messages; hence, I cannot get the feature importances of LightGBM
#gbcvmodeltiny.bestModel.getFeatureImportances(importance_type='split')

### 4.4 Predictions 

#### LightGBM

In [107]:
predict = gbcvmodeltiny.transform(mldata.select('features'))

In [108]:
predict.columns

['features', 'rawPrediction', 'probability', 'prediction']

In [109]:
mldata.select('MachineIdentifier','HasDetections','features')\
      .join(predict.select('features','prediction'),mldata.features==predict.features)\
      .show(5)

+--------------------+-------------+--------------------+--------------------+----------+
|   MachineIdentifier|HasDetections|            features|            features|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|e7aa60177047f9807...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       1.0|
|4c4652d8c2ec536e4...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       1.0|
|3d0a2862336a50e93...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       1.0|
|5793965ea86e97b20...|            0|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
|1c8ed5ff343f2afa1...|            0|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
+--------------------+-------------+--------------------+--------------------+----------+
only showing top 5 rows



In [110]:
resultdf = \
mldata.select('MachineIdentifier','HasDetections','features')\
      .join(predict.select('features','prediction'),mldata.features==predict.features)

In [111]:
resultdf.cache()

DataFrame[MachineIdentifier: string, HasDetections: int, features: vector, features: vector, prediction: double]

In [112]:
resultdf.count()

8939807

In [113]:
resultdf.columns

['MachineIdentifier', 'HasDetections', 'features', 'features', 'prediction']

In [114]:
resultdf.crosstab('HasDetections','prediction').show()

+------------------------+-------+-------+
|HasDetections_prediction|    0.0|    1.0|
+------------------------+-------+-------+
|                       1|      0|4463906|
|                       0|4475901|      0|
+------------------------+-------+-------+



> I guess `LightGBM` is too good; hence, overfit in the given train set. 
> But, randomforest can not even overfit the data. Hence, we may know why Grandient Boosted Trees are a magic word in many ML problems.  

#### RandomForest

In [115]:
rfpredict = rfcvmodeltiny.transform(mldata.select('features'))

In [116]:
mldata.select('MachineIdentifier','HasDetections','features')\
      .join(rfpredict.select('features','prediction'),mldata.features==rfpredict.features)\
      .show(5)

+--------------------+-------------+--------------------+--------------------+----------+
|   MachineIdentifier|HasDetections|            features|            features|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|e7aa60177047f9807...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
|4c4652d8c2ec536e4...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
|3d0a2862336a50e93...|            1|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
|5793965ea86e97b20...|            0|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
|1c8ed5ff343f2afa1...|            0|(383218,[1,2,3,4,...|(383218,[1,2,3,4,...|       0.0|
+--------------------+-------------+--------------------+--------------------+----------+
only showing top 5 rows



In [117]:
rfresultdf = \
mldata.select('MachineIdentifier','HasDetections','features')\
      .join(rfpredict.select('features','prediction'),mldata.features==rfpredict.features)

In [118]:
rfresultdf.crosstab('HasDetections','prediction').show()

+------------------------+-------+-------+
|HasDetections_prediction|    0.0|    1.0|
+------------------------+-------+-------+
|                       1|2573972|1889934|
|                       0|3426111|1049790|
+------------------------+-------+-------+

